# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents
5. Multi-Agents

In [1]:
import json
import pandas as pd
import random
import uvicorn
from pathlib import Path
from datetime import datetime
from sklearn.metrics import accuracy_score
from pydantic import BaseModel, Field
from typing import Any
from fastapi import FastAPI
from io import StringIO
from language_models.agents.chain import AgentChain
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.tools.tool import Tool
from language_models.proxy_client import BTPProxyClient
from language_models.agents.react import ReActAgent
from language_models.tools.earthquake import earthquake_tools
from language_models.tools.current_date import current_date_tool
from language_models.settings import settings
from pprint import pprint

In [2]:
proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.0,
)

In [4]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Sentiment: Positive


In [5]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative."""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

positive


In [6]:
system_prompt = "Take the following movie review and determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Wow! This movie was incredible. The acting was superb, and the plot kept me on the edge of my seat. I highly recommend it!"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

1


In [7]:
system_prompt = "Take the following movie review determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
pprint(response)

("I'm sorry, I am an AI language model and I do not have access to real-time "
 'weather information. I recommend checking a reliable weather website or '
 'using a weather app to get the most accurate and up-to-date forecast for '
 'Seattle.')


In [8]:
system_prompt = """Take the following movie review and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

-1


## 2. Tools

In [9]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12" # answer: $1,289.98

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Total Raw Cost = $1,290.98


In [10]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression.")

calculator_tool = Tool(
    func=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations.",
    args_schema=Calculator
)

print(calculator_tool)

tool name: Calculator, tool description: Use this tool when you want to do calculations., tool input: {{'expression': {{'description': 'A math expression.', 'title': 'Expression', 'type': 'string'}}}}


In [11]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Use a JSON blob to specify a thought, a tool by providing an tool key (tool name) and a tool_input key (tool input).

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "To calculate the total raw cost, you need to add up all the individual costs.",
    "tool": "Calculator",
    "tool_input": {
        "expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"
    }
}


In [12]:
print(calculator(**response["tool_input"]))

1289.98


In [13]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Use a JSON blob to specify a thought, a tool by providing an tool key (tool name) and a tool_input key (tool input).

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}

Observation: tool result
... (this Thought/Tool/Observation can repeat N times)

When you know the answer, use the following JSON format:
{{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": "The final answer to the question",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=json.dumps(response)),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=f"Response of Calculator tool: {calculator(**response['tool_input'])}"),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "I now know the total raw cost.",
    "tool": "Final Answer",
    "tool_input": "The total raw cost is $1289.98."
}


## 3. Chat-based AI Agents

### Earthquake

In [14]:
system_prompt = """You are an United States Geological Survey expert who can answer questions regarding earthquakes and can run forecasts.

Use the current date tool to access the local date and time before using other tools.

Take the following question and answer it as accurately as possible."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=2048,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

earthquake_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{question}",
    task_prompt_variables=["question"],
    tools=earthquake_tools + [current_date_tool],
    output_format=Output,
    iterations=10,
)

In [15]:
response = earthquake_agent.invoke({"question": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"})

18/05/24 14:31:09 INFO Prompt:
How many earthquakes have occurred for the past week with a magnitude of 5 or greater?
18/05/24 14:31:13 INFO Raw response:
{
  "thought": "First, I need to get the current date to calculate the start time for the past week.",
  "tool": "Current Date",
  "tool_input": {}
}
18/05/24 14:31:13 INFO Thought:
First, I need to get the current date to calculate the start time for the past week.
18/05/24 14:31:13 INFO Tool:
Current Date
18/05/24 14:31:13 INFO Tool input:
{}
18/05/24 14:31:13 INFO Tool response:
2024-05-18 14:31:13.199431
18/05/24 14:31:16 INFO Raw response:
{
  "thought": "I need to count the number of earthquakes that have occurred in the past week with a magnitude of 5 or greater.",
  "tool": "Count",
  "tool_input": {
    "start_time": "2024-05-11T14:31:13.199431",
    "min_magnitude": 5
  }
}
18/05/24 14:31:16 INFO Thought:
I need to count the number of earthquakes that have occurred in the past week with a magnitude of 5 or greater.
18/05/24

In [16]:
print(response.final_answer["content"])

There have been 22 earthquakes with a magnitude of 5 or greater in the past week.


In [18]:
response = earthquake_agent.invoke({"question": "Query 10 earthquakes that occurred yesterday and have a magnitude > 3."})

18/05/24 14:33:54 INFO Prompt:
Query 10 earthquakes that occurred yesterday and have a magnitude > 3.
18/05/24 14:34:01 INFO Raw response:
{
  "thought": "To answer this question, I need to query the earthquake data from yesterday with a magnitude greater than 3. I will use the 'Query' tool for this.",
  "tool": "Query",
  "tool_input": {
    "start_time": "2024-05-17T00:00:00",
    "end_time": "2024-05-17T23:59:59",
    "limit": 10,
    "min_magnitude": 3
  }
}
18/05/24 14:34:01 INFO Thought:
To answer this question, I need to query the earthquake data from yesterday with a magnitude greater than 3. I will use the 'Query' tool for this.
18/05/24 14:34:01 INFO Tool:
Query
18/05/24 14:34:01 INFO Tool input:
{'start_time': '2024-05-17T00:00:00', 'end_time': '2024-05-17T23:59:59', 'limit': 10, 'min_magnitude': 3}
18/05/24 14:34:01 INFO Tool response:
{'type': 'FeatureCollection', 'metadata': {'generated': 1716035641000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojs

In [19]:
print(response.final_answer["content"])

Here are 10 earthquakes that occurred yesterday with a magnitude greater than 3:

1. [M 4.7 - 67 km SE of Kirakira, Solomon Islands](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mzb4)
2. [M 4.4 - 64 km ESE of Pondaguitan, Philippines](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mzb2)
3. [M 5.8 - Kermadec Islands, New Zealand](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mzau)
4. [M 4.2 - 268 km W of Adak, Alaska](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mz9e)
5. [M 3.6 - 62 km NNW of Atka, Alaska](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mz9k)
6. [M 4.3 - 40 km WNW of False Pass, Alaska](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mz7b)
7. [M 4.6 - 52 km SW of Puerto Madero, Mexico](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mz64)
8. [M 3.0 - 40 km WSW of Tyonek, Alaska](https://earthquake.usgs.gov/earthquakes/eventpage/ak0246ccfvku)
9. [M 4.9 - southern Mid-Atlantic Ridge](https://earthquake.usgs.

In [20]:
response = earthquake_agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

18/05/24 14:34:38 INFO Prompt:
Can MegaQuakes really happen? Like a magnitude 10 or larger?
18/05/24 14:34:47 INFO Raw response:
{
  "thought": "The user is asking about the possibility of a magnitude 10 or larger earthquake, also known as a 'MegaQuake'. The largest recorded earthquake in history was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would release 31.6 times more energy than the 9.5 earthquake. Such an event would require a fault line capable of accommodating such an event, which does not exist on Earth. Therefore, a magnitude 10 or larger earthquake, while theoretically possible, is incredibly unlikely.",
  "tool": "Final Answer",
  "tool_input": {"content": "The largest recorded earthquake in history was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would release 31.6 times more energy than the 9.5 earthquake. Such an event would require a fault line capable of accommodating such an event, which does not exist on Earth. Therefore, a magnitude 1

In [21]:
pprint(response.final_answer["content"])

('The largest recorded earthquake in history was a magnitude 9.5 in Chile in '
 '1960. A magnitude 10 earthquake would release 31.6 times more energy than '
 'the 9.5 earthquake. Such an event would require a fault line capable of '
 'accommodating such an event, which does not exist on Earth. Therefore, a '
 'magnitude 10 or larger earthquake, while theoretically possible, is '
 'incredibly unlikely.')


## 4. Service-based AI Agents

### Contract Drafting

In [22]:
system_prompt = """Take the follow content and generate a draft of a section for a contract.

Write the content from scratch and make it sound professional.

Respond with the title of the section and the content of the section."""

task_prompt = """Section content:
{section}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class ContractSection(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")

contract_drafting_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["section", "section_name"],
    tools=None,
    output_format=ContractSection,
    iterations=5,
)

In [23]:
def generate_contract(contract_sections: list[str]) -> str:
    sections = []
    for contract_section in contract_sections:
        response = contract_drafting_agent.invoke({"section": contract_section})
        section = response.final_answer["title"] + "\n\n" + response.final_answer["content"]
        sections.append(section)
    return "\n\n".join(sections)

In [24]:
definitions = "Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA."

amendment = """INVOICING AND PAYMENT TERMS
Clause 12.1(ii) of the GMA shall be cancelled and substituted as follow:
[*****]
[*****]
[*****]

Any other provision of Clause 12 shall remain in full force and effect.

PRICE CONDITIONS
(i) Clause 3.2 of the Exhibit 14 of the GMA shall be cancelled and substituted as follow:
“3.2 Technical conditions for prices adjustment
The prices set out in this Exhibit 14 shall be modified every [*****] at the occasion of the invoicing reconciliation pursuant to Clause 11
(“Reconciliation”) if the Standard Operations of the Aircraft, analyzed at the time of the adjustment (all calculations are made with figures corresponding to [*****], change by more or less
[*****] with respect to the estimated values of the same parameters, considered at the time of commencement of the Term.
As from the date this Agreement enters into force, the Parties agree to take into account the following basic operating parameters (the
“Standard Operations”) as a reference for the above calculation:
[*****]
[*****]
[*****]"""

effective_date_and_duration = "Amendment is effective starting on the date of its signature by both Parties."

confidentiality = """Confidential Information released by either of the Parties (the “Disclosing Party”) to the
other Party (the “Receiving Party”) shall not be released in whole or in part to any third party:
- Not to deliver, disclose or publish it to any third party including subsidiary companies and companies having an interest in its capital
- Use Confidential Information solely for the purpose of this Amendment
- Disclose the Confidential Information only to those of its direct employees
- Not to duplicate the Confidential Information nor to copy

Any Confidential Information shall remain the property of the Disclosing Party.

The Receiving Party hereby acknowledges and recognises that Confidential Information is protected by copyright Laws and related
international treaty provisions, as the case may be.

This shall survive termination or expiry of this Amendment for a period of five (5) years following such End Date."""

governing_law = """Pursuant to and in accordance with Section 5-1401 of the New York General Obligations Law.

Arbitration: in the event of a dispute arising out of or relating to this Amendment, including without limitation disputes regarding the
existence, validity or termination of this Amendment (a “Dispute”), either Party may notify such Dispute to the other through service of a
written notice (the “Notice of Dispute”).

Arbitration, and any proceedings, and meetings incidental to or related to the arbitration process, shall take place in New York.

Arbitration shall be kept confidential and the existence of the proceeding and any element.

During any period of negotiation or arbitration, the Parties shall continue to meet their respective obligations.

Notwithstanding any provision of this the Parties may, at any time, seek and decide to settle a Dispute.

Judgment upon any award may be entered in any court having jurisdiction.

Recourse to jurisdictions is expressly excluded except as provided for in the ICC Rules of Conciliation and Arbitration."""

miscellaneous = """Amendment contains the entire agreement between the Parties regarding the subject-matter.

Amendment shall not be varied or modified except by a written document duly signed."""

contract = generate_contract(
    contract_sections=[
        definitions,
        amendment,
        effective_date_and_duration,
        confidentiality,
        governing_law,
        miscellaneous,
    ]
)

18/05/24 14:34:54 INFO Prompt:
Section content:
Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA.
18/05/24 14:35:02 INFO Raw response:
{
  "thought": "The user wants to create a section in a contract that defines the usage of capitalized terms, whether in singular or plural form, and their meaning in the context of the GMA (General Master Agreement). This section is important to ensure clarity and avoid any potential misunderstandings or disputes over the interpretation of the terms used in the contract.",
  "tool": "Final Answer",
  "tool_input": {
    "title": "Interpretation of Capitalized Terms",
    "content": "All capitalized terms, whether expressed in singular or plural form, as used in this Amendment, shall carry the same meaning as defined in the General Master Agreement (GMA). This provision is intended to ensure consistency and clarity in the interpretation and application of these terms throughout the entirety of this co

In [25]:
print(contract)

Interpretation of Capitalized Terms

All capitalized terms, whether expressed in singular or plural form, as used in this Amendment, shall carry the same meaning as defined in the General Master Agreement (GMA). This provision is intended to ensure consistency and clarity in the interpretation and application of these terms throughout the entirety of this contract.

Definition of Terms

Capitalised terms, whether in singular or plural form, used in this Amendment, shall have the same meaning as defined in the General Master Agreement (GMA).

Interpretation of Terms, Invoicing and Payment Terms, and Amendment Effectiveness

INTERPRETATION OF TERMS: Any capitalized terms, whether in singular or plural form, utilized within this Amendment shall retain their original definitions as specified within the General Master Agreement (GMA).

INVOICING AND PAYMENT TERMS: The existing Clause 12.1(ii) of the GMA is hereby nullified and replaced with the following provisions: [*****]. All other stipu

### Sentiment Analysis

In [26]:
df_tweets = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.where(df_tweets.sentiment != 2)
df_tweets["sentiment"] = df_tweets["sentiment"].map({4: 1, 0: 0})
df_tweets_sampled = df_tweets.sample(n=10)
df_tweets_sampled.head()

,sentiment,id,date,query,user,tweet
330773,0,2012390720,Tue Jun 02 20:34:01 PDT 2009,NO_QUERY,sillykimmy,Did u? RE: TanyaAX: I think I just killed ...
502674,0,2187609979,Mon Jun 15 20:32:10 PDT 2009,NO_QUERY,natneagle,@xomollydollyxo natychat also is natywrites ju...
492738,0,2184371114,Mon Jun 15 15:48:43 PDT 2009,NO_QUERY,sixtycigarettes,My phone's nearly dead. Goodbye social life. ...
1167579,1,1980064719,Sun May 31 05:33:33 PDT 2009,NO_QUERY,emboosh,i will be the next johnnyy taylorrr
1141760,1,1977220225,Sat May 30 20:24:59 PDT 2009,NO_QUERY,Shad27,@Vickie6dotcom Can't wait to play a game of Ma...


In [27]:
system_prompt = """Take the following tweet and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class Sentiment(BaseModel):
    sentiment: int = Field(description="The sentiment of the tweet.")

sentiment_analysis_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Tweet:\n{tweet}",
    task_prompt_variables=["tweet"],
    tools=None,
    output_format=Sentiment,
    iterations=2,
)

In [28]:
def classify_sentiment(tweet: str) -> int:
    response = sentiment_analysis_agent.invoke({'tweet': tweet})
    return response.final_answer['sentiment'] or 0

In [29]:
df_tweets_sampled["prediction"] = [classify_sentiment(tweet) for tweet in df_tweets_sampled.tweet]

18/05/24 14:36:28 INFO Prompt:
Tweet:
Did u?     RE: TanyaAX: I think I just killed a rabbit 
18/05/24 14:36:31 INFO Raw response:
{
  "thought": "The tweet seems to be neutral as it's just asking a question about a previous statement. It doesn't express any positive or negative sentiment.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 0}
}
18/05/24 14:36:31 INFO Thought:
The tweet seems to be neutral as it's just asking a question about a previous statement. It doesn't express any positive or negative sentiment.
18/05/24 14:36:31 INFO Final answer:
{'sentiment': 0}
18/05/24 14:36:31 INFO Prompt:
Tweet:
@xomollydollyxo natychat also is natywrites just haven't written since spring break 
18/05/24 14:36:41 INFO Raw response:
{
  "thought": "The tweet seems neutral as it's just sharing information about the user's writing habits. There's no clear positive or negative sentiment.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 1}
}
18/05/24 14:36:41 INFO Thought:
The tw

In [30]:
print(f"Accuracy: {accuracy_score(df_tweets_sampled.sentiment, df_tweets_sampled.prediction)}")

Accuracy: 0.7


### Structuring Unstructured Data

In [31]:
path = Path("./data/jobs")
filenames = [file.name for file in path.iterdir() if file.is_file()]
filenames = random.sample(filenames, 5)

jobs = []
for filename in filenames:
    file_path = path / filename
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        content = file.read()
        jobs.append(content)

In [32]:
system_prompt = """Take the following job and extract data about the job.

Respond with the following extracted data:
- job_title: The job title.
- job_class_no: The job class code.
- job_duties: The duties of the job.
- open_date: When the position was opened. Format: DD-MM-YYYY.
- salary: The salary ranges. Format: 'min salary to max salary'.
- deadline: The application deadline. Format: DD-MM-YYYY.
- application_form: The form of the application (e.g. online, fax, email).
- where_to_apply: The url to apply at or location to send the fax or email address."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    float=0.0,
)

class Job(BaseModel):
    job_title: str = Field(description="The job title.")
    job_class_no: int = Field(description="The job class code.")
    job_duties: str = Field(description="The duties of the job.")
    open_date: str = Field(description="When the position was opened. Format: DD-MM-YYYY.")
    salary: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")
    deadline: str = Field(description="The application deadline. Format: DD-MM-YYYY")
    application_form: str = Field(description="The form of the application (e.g. online, fax, email).")
    where_to_apply: str = Field(description="The url to apply at or location to send the fax or email address.")

job_data_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job}",
    task_prompt_variables=["job"],
    tools=None,
    output_format=Job,
    iterations=10,
)

In [33]:
def extract_jobs(jobs: list[str]) -> pd.DataFrame:
    data = []
    for job in jobs:
        response = job_data_agent.invoke({"job": job})
        data.append(response.final_answer)
    return pd.DataFrame(data)

In [34]:
df_jobs = extract_jobs(jobs)

18/05/24 14:37:16 INFO Prompt:
Job description:



WATER MICROBIOLOGIST
Class Code:       7857
Open Date:  07-25-14
											 Revised:      07-31-14
ANNUAL SALARY

$63,663 to $79,073; $68,611 to $85,232; $78,842 to $97,969
The salary in the Department of Water and Power is $74,896 to 93,041; $90,890 to $112,939

NOTES:

1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.
2. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
3. The current salary range is subject to change. You may confirm the starting salary with the hiring department before accepting a job offer.

DUTIES

A Water Microbiologist uses a variety of microbiological and molecular methods and techniques in performing various analysis, tests, and special studies of microbial activities in aquatic environments; prepares reports and recommendations on the impact of su

In [35]:
df_jobs.head()

,job_title,job_class_no,job_duties,open_date,salary,deadline,application_form,where_to_apply
0,Water Microbiologist,7857,A Water Microbiologist uses a variety of micro...,25-07-2014,"[$63,663 to $79,073, $68,611 to $85,232, $78,8...",07-08-2014,online,http://agency.governmentjobs.com/lacity/defaul...
1,AIRPORT SUPERINTENDENT OF OPERATIONS,7268,An Airport Superintendent of Operations inspec...,18-12-2015,"[$53,682 to $78,467, $66,294 to $96,904, $78,6...",31-12-2015,online,http://agency.governmentjobs.com/lacity/defaul...
2,BUILDING MECHANICAL INSPECTOR,4251,A Building Mechanical Inspector inspects resid...,28-08-2015,"[$83,144, $100,516]",16-05-2019,online,https://www.governmentjobs.com/careers/lacity
3,RECREATION SUPERVISOR,2460,"A Recreation Supervisor plans, organizes and d...",14-10-2016,"[$63,266, $92,498]",03-11-2016,online,https://www.governmentjobs.com/careers/lacity/...
4,SECURITY AIDE,3199,Inspects City facilities for security or safet...,04-09-2015,"[$34,660, $43,075]",17-09-2015,online,http://agency.governmentjobs.com/lacity/defaul...


## 5. Multi-Agents

### Comparing Unstructured Data

In [36]:
def get_job(path: str) -> str:
    with open(path, "r", encoding="utf-8") as file:
        content = file.read()
        return content

job1 = get_job("./data/jobs/ELECTRICAL ENGINEERING ASSOCIATE 7525 093016 REV 100416.txt")
job2 = get_job("./data/jobs/ELECTRICAL MECHANIC 3841 012017.txt")

In [37]:
system_prompt = """Take the following job and extract data about the job.

Respond with the following extracted data:
- job_title: The job title.
- job_duties: The duties of the job.
- salary: The salary ranges. Format: 'min salary to max salary'."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class Job1(BaseModel):
    job1_title: str = Field(description="The job title.")
    job1_duties: str = Field(description="The duties of the job.")
    salary1: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

class Job2(BaseModel):
    job2_title: str = Field(description="The job title.")
    job2_duties: str = Field(description="The duties of the job.")
    salary2: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

job_agent1 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job1}",
    task_prompt_variables=["job1"],
    tools=None,
    output_format=Job1,
    iterations=10,
)

job_agent2 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job2}",
    task_prompt_variables=["job2"],
    tools=None,
    output_format=Job2,
    iterations=10,
)

In [38]:
system_prompt = "Take the following 2 job descriptions and respond with the similarities and differences of the jobs."

task_prompt = """Compare the 2 given job descriptions:

Job 1:
Job title: {job1_title}
Job duties:
{job1_duties}
Salary:
{salary1}


Job 2:
Job title: {job2_title}
Job duties:
{job2_duties}
Salary:
{salary2}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class JobComparison(BaseModel):
    similarities: str = Field(description="The job similarities.")
    differences: str = Field(description="The job differences.")

job_comparison_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["job1_title", "job1_duties", "salary1", "job2_title", "job2_duties", "salary2"],
    tools=None,
    output_format=JobComparison,
    iterations=10,
)

In [39]:
chain = AgentChain(
    chain=[job_agent1, job_agent2, job_comparison_agent],
    chain_variables=["job1", "job2"],
)

In [40]:
response = chain.invoke({"job1": job1, "job2": job2})

18/05/24 14:38:28 INFO Prompt:
Job description:
ELECTRICAL ENGINEERING ASSOCIATE
Class Code:       7525
Open Date:  09-30-16
REVISED: 10-04-16
 (Exam Open to All, including Current City Employees)
ANNUAL SALARY 

$66,231 to $94,252; $74,082 to $105,444; $82,497 to $117,346; and $89,638 to $127,556
The salary in the Department of Water and Power is $77,360 to $96,110; $91,934 to $114,213; $99,722 to $123,881; and $107,156 to 
$133,130

NOTES:

1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.
2. For information regarding reciprocity between City of Los Angeles departments and LADWP, go to: http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
3. The current salary range is subject to change. You may confirm the starting salary with the hiring department before accepting a job offer.

DUTIES

An Electrical Engineering Associate performs professional electrical engineering work in the preparation of designs, plans, specifications

In [41]:
pprint(response.final_answer["similarities"])

('Both jobs are in the electrical field and involve working with electrical '
 'systems and equipment. They both require professional knowledge and skills '
 'in electrical work. Both jobs may involve working in various facilities and '
 'buildings.')


In [42]:
pprint(response.final_answer["differences"])

('The Electrical Engineering Associate is more focused on design, planning, '
 'and technical direction, and may also perform code enforcement functions. '
 'This job has a wider salary range and potentially higher pay. The Electrical '
 'Mechanic, on the other hand, is more focused on the installation and '
 'maintenance of electrical circuits and equipment, and may involve working at '
 'heights, near hazardous materials, and in confined spaces. The salary range '
 'for this job is narrower and potentially lower.')


### Machine Learning Code Generation

In [43]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

Respond with 1 of the following machine learning problems:
- Classification
- Regression
- Clustering
- Time series forecasting"""

task_prompt = """Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
{problem_description}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class ModelingProblem(BaseModel):
    modeling_problem: str = Field(description="The machine learning problem.")

problem_finder_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["problem_description", "dataset_size", "dataset_schema"],
    tools=None,
    output_format=ModelingProblem,
    iterations=5,
)

In [44]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

You can solve machine learning problems for:
- Classification
- Regression
- Clustering
- Time series forecasting

You have access to the following Python libraries:
- pandas
- numpy
- scikit-learn"""

task_prompt = """Given the following machine learning problem, respond with Python code.

Modeling problem: {modeling_problem}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}
First 10 rows of dataset:
{dataset_snippet}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    float=0.0,
)

class AutoMLCode(BaseModel):
    code: str = Field(description="The Python machine learning code.")

ml_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["modeling_problem", "dataset_size", "dataset_schema", "dataset_snippet"],
    tools=None,
    output_format=AutoMLCode,
    iterations=10,
)

In [45]:
ml_chain = AgentChain(
    chain=[problem_finder_agent, ml_agent],
    chain_variables=["problem_description", "dataset_size", "dataset_schema", "dataset_snippet"]
)

In [46]:
info_str = StringIO()
df_tweets.info(buf=info_str)
dataset_schema = info_str.getvalue()

In [47]:
response = ml_chain.invoke({
    "problem_description": "I want to classify the sentiment of tweets.",
    "dataset_size": len(df_tweets),
    "dataset_schema": dataset_schema,
    "dataset_snippet": str(df_tweets.head(10).to_markdown())
})

18/05/24 14:40:23 INFO Prompt:
Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
I want to classify the sentiment of tweets.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

18/05/24 14:40:29 INFO Raw response:
{
  "thought": "Given the problem description and the dataset, the user wants to classify the sentiment of tweets. This is a typical text classification problem, where the goal is to categorize text into predefined classes. In this case, the cla

In [48]:
print(response.final_answer["code"])

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Assuming df is your DataFrame and it's already loaded
X = df['tweet']
y = df['sentiment']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = vectorizer.transform(X_test)

# Create a Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Print a classification report
print(classification_report(y_test, y_pred))


### Forecasting

In [49]:
from datetime import timedelta
from language_models.tools.forecasting import get_earthquakes_data, ml_model

In [50]:
class Forecast(BaseModel):
    start_time: str = Field(None, description='Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')
    end_time: str = Field(None, description='Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')

def forecast(start_time = None, end_time = None):
    if start_time is None:
        start_time = (datetime.now() - timedelta(days=30)).date()
    if end_time is None:
        end_time = (datetime.now().date())
    df = get_earthquakes_data('https://earthquake.usgs.gov/fdsnws/event/1/query?', start_time, end_time)
    df_pred = ml_model.predict(df)
    return {'predictions': df_pred.to_dict(orient='records')}

In [51]:
forecasting_tool = Tool(func=forecast, name='forecast', description='Test forecast model on real-time events.', args_schema=Forecast)

In [52]:
task_prompt = """Take the following question and determine the start and end time to respond with.

Question:
{question}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=256,
    float=0.0,
)

class DateRange(BaseModel):
    start_time: str = Field(description="Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")
    end_time: str = Field(description="Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")

time_wizard_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt=task_prompt,
    task_prompt_variables=["question"],
    tools=[current_date_tool],
    output_format=DateRange,
    iterations=5,
)

In [53]:
forecast_chain = AgentChain(
    chain=[time_wizard_agent, forecasting_tool],
    chain_variables=["question"],
)

In [54]:
response = forecast_chain.invoke({"question": "Run a forecast using the past week as data."})

18/05/24 14:41:09 INFO Prompt:
Take the following question and determine the start and end time to respond with.

Question:
Run a forecast using the past week as data.
18/05/24 14:41:13 INFO Raw response:
{
  "thought": "To answer this question, I need to determine the current date and then calculate the date one week ago.",
  "tool": "Current Date",
  "tool_input": {}
}
18/05/24 14:41:13 INFO Thought:
To answer this question, I need to determine the current date and then calculate the date one week ago.
18/05/24 14:41:13 INFO Tool:
Current Date
18/05/24 14:41:13 INFO Tool input:
{}
18/05/24 14:41:13 INFO Tool response:
2024-05-18 14:41:13.403113
18/05/24 14:41:18 INFO Raw response:
{
  "thought": "The current date is 2024-05-18. To run a forecast using the past week as data, the start time would be 7 days before the current date and the end time would be the current date.",
  "tool": "Final Answer",
  "tool_input": {
    "start_time": "2024-05-11T14:41:13.403113",
    "end_time": "202

In [55]:
print(response.final_answer["forecast"])

{'predictions': [{'time': Timestamp('2024-05-11 14:41:42.310000+0000', tz='UTC'), 'prediction': 1.899396, 'latitude': 39.0701, 'longitude': -119.9166, 'mag': 0.6, 'id': 'nn00877451', 'place': '2 km SE of Glenbrook, Nevada', 'location': '2 km SE of Glenbrook, Nevada'}, {'time': Timestamp('2024-05-11 14:42:13.800000+0000', tz='UTC'), 'prediction': 2.215415, 'latitude': 36.278, 'longitude': -97.50466667, 'mag': 0.96, 'id': 'ok2024jgma', 'place': '4 km W of Lucien, Oklahoma', 'location': '4 km W of Lucien, Oklahoma'}, {'time': Timestamp('2024-05-11 14:42:54.130000+0000', tz='UTC'), 'prediction': 3.709888, 'latitude': 18.5688333333333, 'longitude': -66.7168333333333, 'mag': 2.42, 'id': 'pr71448363', 'place': '10 km N of Arecibo, Puerto Rico', 'location': '10 km N of Arecibo, Puerto Rico'}, {'time': Timestamp('2024-05-11 14:44:30.820000+0000', tz='UTC'), 'prediction': 1.857812, 'latitude': 40.2138333333333, 'longitude': -121.142833333333, 'mag': 2.51, 'id': 'nc75004966', 'place': '1 km ENE o

In [56]:
app = FastAPI()

class Question(BaseModel):
    content: str

class Forecast(BaseModel):
    predictions: list[dict]

@app.get("/forecast")
def forecast(question: Question) -> Forecast:
    response = chain.invoke({"question": question.content})
    return response.final_answer["forecast"]

In [ ]:
if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

### LLM-backed Tools

In [57]:
earthquake_agent.reset()
problem_finder_agent.reset()
ml_agent.reset()

In [58]:
class EarthquakeAgent(BaseModel):
    question: str = Field(description="The question regarding earthquakes.")

def answer_earthquake_questions(question: str) -> Any:
    response = earthquake_agent.invoke({"question": question})
    return response.final_answer

earthquake_agent_tool = Tool(
    func=answer_earthquake_questions,
    name="Earthquake Agent",
    description="Use this tool to answer questions about earthquakes.",
    args_schema=EarthquakeAgent,
)

class MLAgent(BaseModel):
    problem_description: str = Field(description="The user problem.")
    dataset_size: int = Field(description="The size of the dataset."),
    dataset_schema: str = Field(description="The dataset schema or information."),
    dataset_snippet: str = Field(description="The dataset snippet aka the first couple of rows of the dataset.")

def generate_ml_code(problem_description: str, dataset_size: int, dataset_schema: str, dataset_snippet: str) -> Any:
    response = ml_chain.invoke({
        "problem_description": problem_description,
        "dataset_size": dataset_size,
        "dataset_schema": dataset_schema,
        "dataset_snippet": dataset_snippet,
    })
    return response.final_answer

ml_agent_tool = Tool(
    func=generate_ml_code,
    name="ML Agent",
    description="Use this tool to generate machine learning code given a problem.",
    args_schema=MLAgent,
)

In [59]:
system_prompt = """You are an Agent that delegates tasks to other Agents by using the appropriate tools.

Use the Earthquake Agent when the question is about earthquakes.

Use the ML Agent when the user wants you to generate machine learning code."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=4096,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

almighty_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{prompt}",
    task_prompt_variables=["prompt"],
    tools=[earthquake_agent_tool, ml_agent_tool],
    output_format=Output,
    iterations=10,
)

/opt/homebrew/Caskroom/miniforge/base/envs/llm-powered-ai-agents/lib/python3.11/site-packages/pydantic/json_schema.py:2158: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The size of the dataset.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/llm-powered-ai-agents/lib/python3.11/site-packages/pydantic/json_schema.py:2158: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The dataset schema or information.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [60]:
response = almighty_agent.invoke({"prompt": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"})

18/05/24 14:41:34 INFO Prompt:
How many earthquakes have occurred for the past week with a magnitude of 5 or greater?
18/05/24 14:41:37 INFO Raw response:
{
  "thought": "The user is asking about earthquakes. I should use the Earthquake Agent to get this information.",
  "tool": "Earthquake Agent",
  "tool_input": {"question": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"}
}
18/05/24 14:41:37 INFO Thought:
The user is asking about earthquakes. I should use the Earthquake Agent to get this information.
18/05/24 14:41:37 INFO Tool:
Earthquake Agent
18/05/24 14:41:37 INFO Tool input:
{'question': 'How many earthquakes have occurred for the past week with a magnitude of 5 or greater?'}
18/05/24 14:41:37 INFO Prompt:
How many earthquakes have occurred for the past week with a magnitude of 5 or greater?
18/05/24 14:41:41 INFO Raw response:
{
  "thought": "First, I need to get the current date to calculate the start time for the past week.",
  "tool"

In [61]:
print(response.final_answer["content"])

There have been 22 earthquakes with a magnitude of 5 or greater in the past week.


In [62]:
prompt = f"""Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: {len(df_tweets)}
Schema:
{dataset_schema}
First 10 rows of dataset:
{df_tweets.head(10).to_markdown()}"""

response = almighty_agent.invoke({"prompt": prompt})

18/05/24 14:41:54 INFO Prompt:
Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

First 10 rows of dataset:
|    |   sentiment |         id | date                         | query    | user            | tweet                                                                                                               |
|---:|------------:|-----------:|:-----------------------------|:---------|:----------------|:-----------------------------------

In [63]:
print(response.final_answer["content"])

Here is the Python code to train a model that predicts the sentiment of a tweet:

```python
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Drop the columns that are not needed
df = df.drop(['id', 'date', 'query', 'user'], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['sentiment'], test_size=0.2, random_state=42)

# Convert the text data into numerical features
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
```

This code first drops the columns that are not needed for the sentiment pre